In [166]:
import gumpy, pandas, pathlib, numpy, copy

from tqdm import tqdm

First, identify where the `snpit` module is stored relative to this repo so we can read in the master list of lineages `snpit` can identify and the names of their files (`id`)

In [267]:
snpit_path = pathlib.Path('../snpit/lib/')
snpit = pandas.read_csv(snpit_path / 'library.csv')
snpit.set_index('id', inplace=True)
snpit[:3]

,species,lineage,sublineage
id,,,
Indo_Oceanic,M. tuberculosis,Lineage 1,NaN
beijing,M. tuberculosis,Lineage 2,NaN
East_African_Indian,M. tuberculosis,Lineage 3,NaN


Then load in the H37Rv reference genome

In [17]:
reference = gumpy.Genome(snpit_path / 'H37Rv.gbk')

In [18]:
reference

NC_000962
NC_000962.3
Mycobacterium tuberculosis H37Rv, complete genome
4411532 bases
ttgacc...acgtcg
metadata for all genes/loci have been included

We are going to need to compare genes later, so faster to just build once up front (take up to 8 min)

In [167]:
reference_genes = {}
for gene_name in tqdm(reference.genes):
    reference_genes[gene_name] = reference.build_gene(gene_name)

100%|███████████████████████████████████████| 3909/3909 [07:34<00:00,  8.61it/s]


## Verify the fields in the lineage-defining files

First, let's check these files behave as we think by loading the `lineage4` list; if the `nucleotide-index` is the same as the GenBank file it should be 1-based and if the `ref` base is what defines that lineage, then the reference genome should have identical bases at these positions.

In [113]:
file_path = snpit_path / 'lineage4'
df = pandas.read_csv(file_path, sep='\t', names=['nucleotide_index', 'ref'])
df = df.sort_values('nucleotide_index')
print(df[:3], '\n')

new_seq = ''.join(i.lower() for i in df.ref)
print(new_seq, '\n')

mask = numpy.isin(reference.nucleotide_index, df.nucleotide_index )
original_seq = ''. join(i for i in reference.nucleotide_sequence[mask])
print(original_seq, '\n')

if original_seq == new_seq:
    print("identical!")

     nucleotide_index ref
22              15117   C
125             42281   C
150             70267   G 

ccgttcgttgtatctgagtaatattttgtatgctttcgcaacgttaaaagagaatatctatccattagtcggggaaccttacgcctagaggcatcatttccgaaatctataaagacaaaaacccttcgcaaaacattgcgggaacttttaccgagacggactc 

ccgttcgttgtatctgagtaatattttgtatgctttcgcaacgttaaaagagaatatctatccattagtcggggaaccttacgcctagaggcatcatttccgaaatctataaagacaaaaacccttcgcaaaacattgcgggaacttttaccgagacggactc 

identical!


Conversely, a list of definitions for another lineage should be ALL different to the reference

In [114]:
file_path = snpit_path / 'beijing'
df = pandas.read_csv(file_path, sep='\t', names=['nucleotide_index', 'ref'])
df = df.sort_values('nucleotide_index')
print(df[:3], '\n')

new_seq = ''.join(i.lower() for i in df.ref)
print(new_seq + '\n')

mask = numpy.isin(reference.nucleotide_index, df.nucleotide_index)
original_seq = ''. join(i for i in reference.nucleotide_sequence[mask])
print(original_seq + '\n')

counter = 0
for (i,j) in zip(original_seq, new_seq):
    if i == j:
        counter += 1
if counter == 0:
    print("all different!")

    nucleotide_index ref
15             11820   G
34             14861   T
41             16119   A 

gtactcgggtctatcttagtgcgcatatacccttattatctgatggcggagtatgtcggaactcgtttgatcgcccatgcttaaccgatgccaatgcgtgggtctaattgagccgatttgtgcgctcgcccgactatagtctagctcgtaaccccctactgagatagtcatagcatagttgcacttaaccacaaacggtaattgttataggtacccgttaaatgctagcaaacgaaccctgttaaaagtgatatcacatttagcccatagaatcctcatcaatatccagtagccgg

cgcgcgccagtcgcacggcgcactccgcgggtcgggcgctctgcccatagactcagtttccgcgaccctgcactaagcaacccgtatcctaaggcagtctaccggggggcgataagccccccaaacatttaaggccgcccagctacgaccgtgggtgctctgagggactccgcgggcccgtaggcgcgaagtgggaaaccgccagcccgttcgattaccgcgcctcgctgctaagggtacaccgcggcgcgcggggggacccaggggggacgctactggtggcgcgggtcgtgtct

all different!


## Test on a single linege

Make a `deepcopy` of the `reference` so we can modify it

In [255]:
sample = copy.deepcopy(reference)

let's test using `Lineage 2`

In [256]:
lineage_name = 'beijing'

In [257]:
file_path = snpit_path / lineage_name
df = pandas.read_csv(file_path, sep='\t', names=['nucleotide_index', 'ref'])
df = df.sort_values('nucleotide_index')
df[:3]

,nucleotide_index,ref
15,11820,G
34,14861,T
41,16119,A


First, let's produce a `VARIANTS` style table with the nucleotide variation

In [258]:
mask = numpy.isin(sample.nucleotide_index, df.nucleotide_index)
sample.nucleotide_sequence[mask] = df.ref.str.lower()
gdiff = reference - sample
gdiff.variants[:5]

array(['11820c>g', '14861g>t', '16119c>a', '25610g>c', '35608c>t'],
      dtype='<U10')

In [259]:
VARIANTS = pandas.DataFrame(gdiff.variants, columns=['VARIANT'])
VARIANTS[:3]

,VARIANT
0,11820c>g
1,14861g>t
2,16119c>a


Now, let's work out what genes those `variants` are in (some will be intergenic), recalculate the `sample` genes and produce `MUTATION` style tables

In [260]:
mask = numpy.isin(sample.stacked_nucleotide_index, df.nucleotide_index)

mutated_genes = numpy.unique(sample.stacked_gene_name[mask])

print("There are " + str(len(mutated_genes)) + " genes that are different in this lineage")
mutated_genes[:5]

There are 266 genes that are different in this lineage


array(['', 'PE14', 'PE16', 'PE23', 'PE24'], dtype='<U20')

In [261]:
rows = []
for gene_name in tqdm(mutated_genes):
    if gene_name != '':
        gene2 = sample.build_gene(gene_name)
        mdiff = reference_genes[gene_name] - gene2
        for mutation in mdiff.mutations:
            rows.append([lineage_name,gene_name, mutation])

100%|█████████████████████████████████████████| 266/266 [00:31<00:00,  8.33it/s]


In [264]:
MUTATIONS = pandas.DataFrame(rows, columns = ['id', 'GENE', 'MUTATION'])
MUTATIONS['id'] = lineage_name
MUTATIONS[:3]

,id,GENE,MUTATION
0,beijing,PE14,A106A
1,beijing,PE16,A96A
2,beijing,PE23,A344T


## Putting it all together

Now that we know what are in the files, and we've tested it, let's put it all together.

The below takes about 20 min to run.

In [269]:
variant_rows = []

VARIANTS = None
MUTATIONS = None

for lineage_name in tqdm(snpit.index):
    
#     print(lineage_name)
    
    sample = copy.deepcopy(reference)
    
    file_path = snpit_path / lineage_name
    snpit_lineage_definition = pandas.read_csv(file_path, sep='\t', names=['nucleotide_index', 'ref'])
    snpit_lineage_definition = snpit_lineage_definition.sort_values('nucleotide_index')
    
    mask = numpy.isin(sample.nucleotide_index, snpit_lineage_definition.nucleotide_index)
    sample.nucleotide_sequence[mask] = snpit_lineage_definition.ref.str.lower()
    gdiff = reference - sample

    df = pandas.DataFrame(gdiff.variants, columns=['VARIANT'])
    df['id'] = lineage_name

    if VARIANTS is None:
        VARIANTS = copy.deepcopy(df)
    else:
        VARIANTS = VARIANTS.append(df)
        
    # Now, let's work out what genes those are in, recalculate them and produce `MUTATION` style tables
    mask = numpy.isin(sample.stacked_nucleotide_index, snpit_lineage_definition.nucleotide_index)

    mutated_genes = numpy.unique(sample.stacked_gene_name[mask])

#     print("There are " + str(len(mutated_genes)) + " genes that are different in this lineage")
    
    rows = []
    for gene_name in mutated_genes:
        if gene_name != '':
            gene2 = sample.build_gene(gene_name)
            mdiff = reference_genes[gene_name] - gene2
            for mutation in mdiff.mutations:
                rows.append([lineage_name,gene_name, mutation])

    df = pandas.DataFrame(rows, columns = ['id', 'GENE', 'MUTATION'])
    df['id'] = lineage_name
    
    if MUTATIONS is None:
        MUTATIONS = copy.deepcopy(df)
    else:
        MUTATIONS = MUTATIONS.append(df)



100%|███████████████████████████████████████████| 26/26 [18:07<00:00, 41.85s/it]


In [270]:
VARIANTS[:3]

,VARIANT,id
0,6112g>c,Indo_Oceanic
1,8452c>t,Indo_Oceanic
2,13298g>c,Indo_Oceanic


In [271]:
MUTATIONS[:3]

,id,GENE,MUTATION
0,Indo_Oceanic,PE4,K164N
1,Indo_Oceanic,PE_PGRS11,G280R
2,Indo_Oceanic,PE_PGRS32,A332T


So that we can join back to the `snpit` master file, let's index

In [272]:
VARIANTS.set_index('id',inplace=True)
MUTATIONS.set_index('id',inplace=True)

Now let's tidy up the tables and write them to disc

In [273]:
MUTATIONS = MUTATIONS.join(snpit)
MUTATIONS.reset_index(inplace=True)
MUTATIONS.rename(columns = {'id': 'SNPIT_ID', 'species':'SPECIES', 'lineage':'LINEAGE', 'sublineage':'SUBLINEAGE'}, inplace=True)
MUTATIONS = MUTATIONS[['SNPIT_ID', 'SPECIES', 'LINEAGE', 'SUBLINEAGE', 'GENE', 'MUTATION']]
MUTATIONS.to_csv('tables/MUTATIONS.csv.gz')
MUTATIONS.to_pickle('tables/MUTATIONS.pkl.gz')
MUTATIONS[:3]

,SNPIT_ID,SPECIES,LINEAGE,SUBLINEAGE,GENE,MUTATION
0,Dassie,Dassie bacillus (ex Procavia capensis),NaN,NaN,PE_PGRS11,T469N
1,Dassie,Dassie bacillus (ex Procavia capensis),NaN,NaN,PE_PGRS11,R512L
2,Dassie,Dassie bacillus (ex Procavia capensis),NaN,NaN,PE_PGRS11,P518L


In [274]:
VARIANTS = VARIANTS.join(snpit)
VARIANTS.reset_index(inplace=True)
VARIANTS.rename(columns = {'id': 'SNPIT_ID', 'species':'SPECIES', 'lineage':'LINEAGE', 'sublineage':'SUBLINEAGE'}, inplace=True)
VARIANTS = VARIANTS[['SNPIT_ID', 'SPECIES', 'LINEAGE', 'SUBLINEAGE', 'VARIANT']]
VARIANTS.to_csv('tables/VARIANTS.csv.gz')
VARIANTS.to_pickle('tables/VARIANTS.pkl.gz')
VARIANTS[:3]

,SNPIT_ID,SPECIES,LINEAGE,SUBLINEAGE,VARIANT
0,Dassie,Dassie bacillus (ex Procavia capensis),NaN,NaN,4087g>t
1,Dassie,Dassie bacillus (ex Procavia capensis),NaN,NaN,5073g>a
2,Dassie,Dassie bacillus (ex Procavia capensis),NaN,NaN,19052c>g
